# Mini-Project I
During this project, we will practice handling of complex lists and dictionaries in Python. Plus, we will learn how to work with API documentation. Don't be afraid to search for the information in the [**documentation**](https://api.tfl.gov.uk/swagger/ui/index.html?url=/swagger/docs/v1#!/AccidentStats/AccidentStats_Get).

Let's go to the tasks, we have some parsing to do :)!!

In [22]:
import requests
from IPython.display import JSON

In [23]:
# import packages we need (remember what packages we used yesterday during the API session)
# IMPORT HERE

## Demo

In [24]:
# URL
url = "https://api.tfl.gov.uk/AirQuality"

In [25]:
# We send the request to the API
# NOTE: if you don't have your APP_KEY, run this without the url_append
res = requests.get(url)

In [26]:
# We can check if the request was successful
res.status_code

200

In [27]:
# We can export the information that was returned using method .json()
data = res.json()

## Task
Parse the dictionary and print the AirQuality predictions for tomorrow

In [28]:
print(data['currentForecast'][1]['forecastSummary'])

Low air pollution forecast valid from Saturday 19 September to end of Monday 21 September GMT


## Task
What are the different modes of transport which are operated by Transfer for London? How many of modes do they have?

Print the list with different modes of transport, plus their count. Example output:
```
[bus, cable-car,.....]
Number of different modes of transport is: xyz
```

We need to search the documentation for correct request.

In [29]:
#set url
url_mode = 'https://api.tfl.gov.uk/Journey/Meta/Modes'

#send request to get all kinds of modes
res_mode = requests.get(url_mode)
print(res_mode.status_code)

200


In [30]:
transport = res_mode.json()
#collect all modes to a single list
modes = [types['modeName'] for types in transport]

#print modes
print(modes)
print("\nNumber of different modes of transport is: {}".format(len(modes)))

['black-cab-as-customer', 'black-cab-as-driver', 'bus', 'cable-car', 'coach', 'cycle', 'cycle-hire', 'dlr', 'electric-car', 'goods-vehicle-as-driver', 'interchange-keep-sitting', 'interchange-secure', 'international-rail', 'motorbike-scooter', 'national-rail', 'overground', 'plane', 'private-car', 'private-coach-as-customer', 'private-coach-as-driver', 'private-hire-as-customer', 'private-hire-as-driver', 'replacement-bus', 'river-bus', 'river-tour', 'taxi', 'tflrail', 'tram', 'tube', 'walking']

Number of different modes of transport is: 30


## Task
How many BikePoints in London are operated by Transfor for London? How many docks are in **all** BikePoints? There is the information for empty and full docks for each BikePoint.

In [31]:
url_bike_points = 'https://api.tfl.gov.uk/BikePoint'
res_bike_points = requests.get(url_bike_points)
print(res_bike_points.status_code)

200


In [32]:
bike_points = res_bike_points.json() #parse to dict

#print the length of bike_points dict
print("BikePoints in London are operated by Transfor for London: {}".format(len(bike_points)))

num_docks = 0 #initalize counter

for bike_point in bike_points:
    for prop in bike_point['additionalProperties']:
        #check if the it is the right list
        if prop['key'] == "NbDocks":
            num_docks += int(prop['value']) #add the number of docks to counter
            
#print the result               
print("Number of Docks in all Bike points: {}".format(num_docks))

BikePoints in London are operated by Transfor for London: 784
Number of Docks in all Bike points: 20853


## Task
How many tube and bus lines are in London? Print names of all tube lines.

In [33]:
#get tubes and bus lines
params = 'tube,bus' #set parameters
url_lines = 'https://api.tfl.gov.uk/Line/Mode/{}'.format(params)
res_lines = requests.get(url_lines)

#check status
print(res_lines.status_code)

200


In [34]:
tube_lines = [lines['name'] for lines in res_lines.json() if lines["modeName"] == "tube"] # collect all the names of the tube lines
bus_lines = [lines['name'] for lines in res_lines.json() if lines["modeName"] == "bus"] # collect all the names of the bus lines


print("Number of all tube lines in London: {} \nNumber of all bus lines in London: {}".format(len(tube_lines),len(bus_lines)))
print("Name of all tube lines: {}".format(tube_lines))

Number of all tube lines in London: 11 
Number of all bus lines in London: 681
Name of all tube lines: ['Bakerloo', 'Central', 'Circle', 'District', 'Hammersmith & City', 'Jubilee', 'Metropolitan', 'Northern', 'Piccadilly', 'Victoria', 'Waterloo & City']


## Task
How many station has `victoria` line?

In [35]:
params = 'victoria' #set parameters
url_station = 'https://api.tfl.gov.uk/Line/{}/StopPoints'.format(params) 
res_station = requests.get(url_station)
print(res_station.status_code)

200


In [36]:
print("Number of stations along victoria line: {}".format(len(res_station.json())))

Number of stations along victoria line: 16


## Task
Plan the journey from Heathrow Airport to Tower Bridge using Public Transport, Taxi or Bike? Which way is the fastest? Example output:
```
Planned duration:
Public Transport: x minutes
Bike: y minutes
```

We need to search the documentation for correct requests.

In [37]:
# API key
api_file = open(".api-key.txt", "r")
api_key = api_file.readline()
api_file.close()

#set destination and origin
origin = 'Heathrow Airport'
dest = 'London Tower Bridge'

# set parameters and url
url = "https://maps.googleapis.com/maps/api/distancematrix/json?"
params = "units=metric&origins={origin}&destinations={dest}&key={api}&".format(origin=origin,dest= dest,api=api_key)

#set a list of modes
travel_modes = "transit bicycling driving".split()

#loop through travel_modes and save them to responses dict
responses = {}
for mode in travel_modes:
    responses[mode] = {
                        "time":requests.get(url+params+"mode="+mode).json()["rows"][0]["elements"][0]["duration"]["text"],
                        "seconds":requests.get(url+params+"mode="+mode).json()["rows"][0]["elements"][0]["duration"]["value"]
                      }


In [38]:
#print all of the travel modes and the duration 
print("Driving: {} \nPublic Transport: {} \nBike: {}"\
      .format(responses['driving']['time'],
              responses['transit']['time'], 
              responses['bicycling']['time']))    

Driving: 53 mins 
Public Transport: 1 hour 9 mins 
Bike: 1 hour 33 mins


In [39]:
#get the keys that have the min second values
min_value_keys = [key for key,val in responses.items() if val["seconds"] == min(mode["seconds"] 
                                                                                for mode in responses.values())]

#print the fastest ways to travel
print("Fast way(s) to travel: {}".format(",".join(min_value_keys)))    

Fast way(s) to travel: driving
